In [120]:
from marketsimulator.gateway import Gateway
import datetime
import time
import numpy as np
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [121]:
%pip install plotly
import plotly.graph_objects as go

Note: you may need to restart the kernel to use updated packages.


In [122]:
class SimplePOV():
    def __init__(self, is_buy, target_pov, lmtpx, qty, sweep_max):
        self.is_buy = is_buy
        self.target_pov = target_pov
        self.lmtpx = lmtpx
        self.qty = qty
        self.cumqty = 0
        self.pov = 0.
        self.sweep_max = sweep_max
        self.done = False                

    def _target_px(self, gtw):
        if self.is_buy:
            max_px = gtw.ob.get_new_price(gtw.ob.best_ask[0], self.sweep_max)
            target_px = min(self.lmtpx, max_px)
        else:
            min_px = gtw.ob.get_new_price(gtw.ob.best_bid[0], -self.sweep_max)            
            target_px = max(self.lmtpx, min_px)
        return target_px        

    def eval_and_act(self, gtw):                    
        
        if gtw.ob.my_cumvol >= self.qty:
            self.done = True
            return False
        
        if gtw.ob.my_pov < self.target_pov:
            target_vol = int(gtw.ob.cumvol * self.target_pov) - gtw.ob.my_cumvol
            next_vol = min(self.qty-gtw.ob.my_cumvol, target_vol)            
            gtw.queue_my_new(is_buy=self.is_buy,
                            qty=next_vol,
                            price=self._target_px(gtw))        
            return True
        else:
            return False

In [134]:
# =============================================================================
# TEST SIMPLE POV ALGO
# =============================================================================

gtw = Gateway(ticker='san',
             date=datetime.date(2019,5,23),        
             start_h= 9,
             end_h= 10,        
             latency= 20_000,
             max_impact=20,
             resilience=0.5)
    

pov_algo = SimplePOV(is_buy=True, target_pov=0.2, lmtpx=np.Inf,
                       qty=int(1e7), sweep_max=1)


In [135]:

end_time =datetime.datetime(2019, 5, 23) + timedelta(0, 9.5*3600)
hist_bidask = list()

t = time.time()
ob_nord = gtw.ob_nord-1
while (not pov_algo.done) and (gtw.ob_time < end_time):
    pov_algo.eval_and_act(gtw)
    gtw.tick()
    hist_bidask.append([gtw.ob.best_bid[0],
                        gtw.ob.best_ask[0],
                        gtw.ob_time])    
    

In [136]:
## MARKET BIDASK
bidask = pd.DataFrame(hist_bidask, columns=['bid', 'ask', 'timestamp'])
bidask.set_index(bidask.timestamp, inplace=True)
# MARKET TRADES 
trades = pd.DataFrame(gtw.ob.trades).loc[:gtw.ob.ntrds-1]
trades.set_index(trades.timestamp, inplace=True)
# MY TRADES 
my_trades = pd.DataFrame(gtw.ob.my_trades).loc[:gtw.ob.my_ntrds-1]
my_trades.set_index(my_trades.timestamp, inplace=True)

start_t = bidask.index[0]
win_size = 5000

end_t = start_t + timedelta(0, win_size)
subest_bidask = bidask.loc[start_t:end_t]
subtrades = trades.loc[start_t:end_t]
mysubtrades = my_trades.loc[start_t:end_t]


In [137]:
## MARKET BIDASK
greenc = '#009900'
redc = '#cc0000'

fig = go.Figure()
fig.add_trace(go.Scatter(x=subbidask.index,
                         y=subbidask.bid,
                         line=dict(color=greenc),
                         name='bid price'))
fig.add_trace(go.Scatter(x=subbidask.index,
                         y=subbidask.ask,
                         line=dict(color=redc),
                         name='ask price'))

idx_buy_init = np.where(subtrades.buy_init * (subtrades.agg_ord > 0))
idx_sell_init = np.where(np.logical_not(subtrades.buy_init) * \
                         (subtrades.agg_ord > 0))
buy_init_trd = subtrades.iloc[idx_buy_init]['price']
sell_init_trd = subtrades.iloc[idx_sell_init]['price']

my_trd = mysubtrades['price']    

fig.add_trace(go.Scatter(x=buy_init_trd.index,
                         y=buy_init_trd.values,
                         mode='markers',
                         marker=dict(color=greenc,
                                     symbol='triangle-up',
                                     size=6),                        
                         name='aggressive buys'
             ))

fig.add_trace(go.Scatter(x=sell_init_trd.index,
                         y=sell_init_trd.values,
                         mode='markers',
                         marker=dict(color=redc,
                                     symbol='triangle-down',
                                     size=6),                        
                         name='aggressive sells'
             ))

fig.add_trace(go.Scatter(x=my_trd.index,
                         y=my_trd.values,
                         mode='markers',
                         marker=dict(color='blue',                                     
                                     size=3),                        
                         name='my trades'
             ))


fig.show()


